In [1]:
!wget -O "/tmp/vocab.txt" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking/vocab.txt"
!wget -O "/tmp/train.tfrecords" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking/ELWC/train.tfrecords"
!wget -O "/tmp/test.tfrecords" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking//ELWC/test.tfrecords"

#https://colab.research.google.com/github/tensorflow/ranking/blob/master/tensorflow_ranking/examples/handling_sparse_features.ipynb#scrollTo=45WYaJNaGfLM

--2020-04-30 20:54:47--  http://ciir.cs.umass.edu/downloads/Antique/tf-ranking/vocab.txt
Resolving ciir.cs.umass.edu (ciir.cs.umass.edu)... 128.119.246.154
Connecting to ciir.cs.umass.edu (ciir.cs.umass.edu)|128.119.246.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘/tmp/vocab.txt’

/tmp/vocab.txt      100%[===================>] 226.08K   989KB/s    in 0.2s    

2020-04-30 20:54:48 (989 KB/s) - ‘/tmp/vocab.txt’ saved [231508/231508]

--2020-04-30 20:54:48--  http://ciir.cs.umass.edu/downloads/Antique/tf-ranking/ELWC/train.tfrecords
Resolving ciir.cs.umass.edu (ciir.cs.umass.edu)... 128.119.246.154
Connecting to ciir.cs.umass.edu (ciir.cs.umass.edu)|128.119.246.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156410796 (149M)
Saving to: ‘/tmp/train.tfrecords’

/tmp/train.tfrecord 100%[===================>] 149.16M  20.8MB/s    in 7.0s    

2020-04-30 20:54:55 (21.3 MB/s) - ‘/tmp/train.tfreco

In [2]:
print('Installing TensorFlow 2.1.0. This will take a minute, ignore the warnings.')
!pip install -q tensorflow==2.1.0
import tensorflow as tf
tf.compat.v1.enable_eager_execution()


# This is needed for tensorboard compatibility.
!pip uninstall -y grpcio
!pip install -q grpcio>=1.24.3
import six
import os
import numpy as np

try:
    import tensorflow_ranking as tfr
except ImportError:
    !pip install -q tensorflow_ranking
    import tensorflow_ranking as tfr

Installing TensorFlow 2.1.0. This will take a minute, ignore the warnings.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -m pip install --upgrade pip' command.
Found existing installation: grpcio 1.28.1
Uninstalling grpcio-1.28.1:
  Successfully uninstalled grpcio-1.28.1
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -m pip install --upgrade pip' command.


In [3]:
# Store the paths to files containing training and test instances.
_TRAIN_DATA_PATH = "/tmp/train.tfrecords"
_TEST_DATA_PATH = "/tmp/test.tfrecords"

# Store the vocabulary path for query and document tokens.
_VOCAB_PATH = "/tmp/vocab.txt"

# The maximum number of documents per query in the dataset.
# Document lists are padded or truncated to this size.
_LIST_SIZE = 50

# The document relevance label.
_LABEL_FEATURE = "relevance"

# Padding labels are set negative so that the corresponding examples can be
# ignored in loss and metrics.
_PADDING_LABEL = -1

# Learning rate for optimizer.
_LEARNING_RATE = 0.05

# Parameters to the scoring function.
_BATCH_SIZE = 32
_HIDDEN_LAYER_DIMS = ["64", "32", "16"]
_DROPOUT_RATE = 0.8
_GROUP_SIZE = 1  # Pointwise scoring.

# Location of model directory and number of training steps.
_MODEL_DIR = "Model/ranking_model_dir"
_NUM_TRAIN_STEPS = 15 * 1000

In [4]:
_EMBEDDING_DIMENSION = 20

def context_feature_columns():
    """Returns context feature names to column definitions."""
    sparse_column = tf.feature_column.categorical_column_with_vocabulary_file(
      key="query_tokens",
      vocabulary_file=_VOCAB_PATH)
    query_embedding_column = tf.feature_column.embedding_column(
      sparse_column, _EMBEDDING_DIMENSION)
    return {"query_tokens": query_embedding_column}

def example_feature_columns():
    """Returns the example feature columns."""
    sparse_column = tf.feature_column.categorical_column_with_vocabulary_file(
      key="document_tokens",
      vocabulary_file=_VOCAB_PATH)
    document_embedding_column = tf.feature_column.embedding_column(
      sparse_column, _EMBEDDING_DIMENSION)
    return {"document_tokens": document_embedding_column}

def input_fn(path, num_epochs=None):
    context_feature_spec = tf.feature_column.make_parse_example_spec(
        context_feature_columns().values())
    label_column = tf.feature_column.numeric_column(
        _LABEL_FEATURE, dtype=tf.int64, default_value=_PADDING_LABEL)
    example_feature_spec = tf.feature_column.make_parse_example_spec(
        list(example_feature_columns().values()) + [label_column])
    dataset = tfr.data.build_ranking_dataset(
        file_pattern=path,
        data_format=tfr.data.ELWC,
        batch_size=_BATCH_SIZE,
        list_size=_LIST_SIZE,
        context_feature_spec=context_feature_spec,
        example_feature_spec=example_feature_spec,
        reader=tf.data.TFRecordDataset,
        shuffle=False,
        num_epochs=num_epochs)

    features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    label = tf.squeeze(features.pop(_LABEL_FEATURE), axis=2)
    label = tf.cast(label, tf.float32)
    print(label)
    print(num_epochs)
    print(path)
    
    return features, label

def make_transform_fn():
    def _transform_fn(features, mode):
        """Defines transform_fn."""
        context_features, example_features = tfr.feature.encode_listwise_features(
            features=features,
            context_feature_columns=context_feature_columns(),
            example_feature_columns=example_feature_columns(),
            mode=mode,
            scope="transform_layer")

        return context_features, example_features
    return _transform_fn

def make_score_fn():
    """Returns a scoring function to build `EstimatorSpec`."""

    def _score_fn(context_features, group_features, mode, params, config):
        """Defines the network to score a group of documents."""
        with tf.compat.v1.name_scope("input_layer"):
            context_input = [
              tf.compat.v1.layers.flatten(context_features[name])
              for name in sorted(context_feature_columns())
          ]
            group_input = [
                tf.compat.v1.layers.flatten(group_features[name])
                  for name in sorted(example_feature_columns())
          ]
            input_layer = tf.concat(context_input + group_input, 1)

            is_training = (mode == tf.estimator.ModeKeys.TRAIN)
            cur_layer = input_layer
            cur_layer = tf.compat.v1.layers.batch_normalization(
                cur_layer,
                training=is_training,
                momentum=0.99)

        for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
            cur_layer = tf.compat.v1.layers.dense(cur_layer, units=layer_width)
            cur_layer = tf.compat.v1.layers.batch_normalization(
                cur_layer,
                training=is_training,
                momentum=0.99)
            cur_layer = tf.nn.relu(cur_layer)
            cur_layer = tf.compat.v1.layers.dropout(
                inputs=cur_layer, rate=_DROPOUT_RATE, training=is_training)
        logits = tf.compat.v1.layers.dense(cur_layer, units=_GROUP_SIZE)
        return logits

    return _score_fn

def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
    A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
    metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
    })

    return metric_fns

def _train_op_fn(loss):
    """Defines train op used in ranking head."""
    update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
    minimize_op = optimizer.minimize(
      loss=loss, global_step=tf.compat.v1.train.get_global_step())
    train_op = tf.group([update_ops, minimize_op])
    return train_op

def train_and_eval_fn():
    """Train and eval function used by `tf.estimator.train_and_evaluate`."""
    run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=1000)
    ranker = tf.estimator.Estimator(
      model_fn=model_fn,
      model_dir=_MODEL_DIR,
      config=run_config)

    train_input_fn = lambda: input_fn(_TRAIN_DATA_PATH)
    eval_input_fn = lambda: input_fn(_TEST_DATA_PATH, num_epochs=1)

    train_spec = tf.estimator.TrainSpec(
      input_fn=train_input_fn, max_steps=_NUM_TRAIN_STEPS)
    eval_spec =  tf.estimator.EvalSpec(
          name="eval",
          input_fn=eval_input_fn,
          throttle_secs=15)
    return (ranker, train_spec, eval_spec)


In [5]:
_LOSS = tfr.losses.RankingLossKey.APPROX_NDCG_LOSS
loss_fn = tfr.losses.make_loss_fn(_LOSS)

optimizer = tf.compat.v1.train.AdagradOptimizer(
    learning_rate=_LEARNING_RATE)

ranking_head = tfr.head.create_ranking_head(
      loss_fn=loss_fn,
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

model_fn = tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          transform_fn=make_transform_fn(),
          group_size=_GROUP_SIZE,
          ranking_head=ranking_head)

! rm -rf "Model/ranking_model_dir"  # Clean up the model directory.
ranker, train_spec, eval_spec = train_and_eval_fn()
tf.estimator.train_and_evaluate(ranker, train_spec, eval_spec)

W0430 20:55:07.158591 4514137536 model_fn.py:630] Estimator's model_fn (<function _make_model_fn.<locals>._model_fn at 0x133978620>) includes params argument, but params are not passed to Estimator.
W0430 20:55:07.212261 4514137536 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0430 20:55:07.221784 4514137536 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow_core/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Var

Tensor("Cast:0", shape=(32, 50), dtype=float32, device=/device:CPU:0)
None
/tmp/train.tfrecords


W0430 20:55:09.599272 4514137536 deprecation.py:323] From <ipython-input-4-7d539fe49ece>:69: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Flatten instead.
W0430 20:55:09.601973 4514137536 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow_core/python/layers/core.py:332: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W0430 20:55:09.694931 4514137536 deprecation.py:323] From <ipython-input-4-7d539fe49ece>:82: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult

Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords


W0430 20:56:22.934764 4514137536 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords
Tensor("Cast:0", shape=(None, 50), dtype=float32, device=/device:CPU:0)
1
/tmp/test.tfrecords


({'labels_mean': 1.9630322,
  'logits_mean': 2.0652447,
  'loss': -0.8387331,
  'metric/ndcg@1': 0.7264286,
  'metric/ndcg@10': 0.85479635,
  'metric/ndcg@3': 0.77445656,
  'metric/ndcg@5': 0.80803823,
  'global_step': 15000},
 [])

In [6]:
context_feature_spec = tf.feature_column.make_parse_example_spec(
        context_feature_columns().values())
label_column = tf.feature_column.numeric_column(
        _LABEL_FEATURE, dtype=tf.int64, default_value=_PADDING_LABEL)
example_feature_spec = tf.feature_column.make_parse_example_spec(
        list(example_feature_columns().values()) + [label_column])
dataset = tfr.data.build_ranking_dataset(
        file_pattern=_TRAIN_DATA_PATH,
        data_format=tfr.data.ELWC,
        batch_size=_BATCH_SIZE,
        list_size=_LIST_SIZE,
        context_feature_spec=context_feature_spec,
        example_feature_spec=example_feature_spec,
        reader=tf.data.TFRecordDataset,
        shuffle=False,
        num_epochs=None)

features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
label = tf.squeeze(features.pop(_LABEL_FEATURE), axis=2)
label = tf.cast(label, tf.float32)

In [7]:
label

<tf.Tensor: shape=(32, 50), dtype=float32, numpy=
array([[ 1.,  2.,  3., ..., -1., -1., -1.],
       [ 3.,  3.,  2., ..., -1., -1., -1.],
       [ 1.,  3.,  2., ..., -1., -1., -1.],
       ...,
       [ 2.,  1.,  3., ..., -1., -1., -1.],
       [ 2.,  3.,  1., ..., -1., -1., -1.],
       [ 2.,  3.,  3., ..., -1., -1., -1.]], dtype=float32)>

In [8]:
print(features["query_tokens"])

SparseTensor(indices=tf.Tensor(
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 0  3]
 [ 0  4]
 [ 0  5]
 [ 0  6]
 [ 0  7]
 [ 0  8]
 [ 0  9]
 [ 0 10]
 [ 0 11]
 [ 0 12]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 1  3]
 [ 1  4]
 [ 1  5]
 [ 1  6]
 [ 1  7]
 [ 1  8]
 [ 1  9]
 [ 1 10]
 [ 1 11]
 [ 1 12]
 [ 1 13]
 [ 2  0]
 [ 2  1]
 [ 2  2]
 [ 2  3]
 [ 2  4]
 [ 2  5]
 [ 2  6]
 [ 2  7]
 [ 3  0]
 [ 3  1]
 [ 3  2]
 [ 3  3]
 [ 3  4]
 [ 3  5]
 [ 3  6]
 [ 3  7]
 [ 4  0]
 [ 4  1]
 [ 4  2]
 [ 4  3]
 [ 4  4]
 [ 4  5]
 [ 4  6]
 [ 4  7]
 [ 4  8]
 [ 4  9]
 [ 5  0]
 [ 5  1]
 [ 5  2]
 [ 5  3]
 [ 5  4]
 [ 5  5]
 [ 5  6]
 [ 5  7]
 [ 5  8]
 [ 6  0]
 [ 6  1]
 [ 6  2]
 [ 6  3]
 [ 6  4]
 [ 6  5]
 [ 6  6]
 [ 6  7]
 [ 6  8]
 [ 7  0]
 [ 7  1]
 [ 7  2]
 [ 7  3]
 [ 7  4]
 [ 7  5]
 [ 7  6]
 [ 7  7]
 [ 7  8]
 [ 7  9]
 [ 7 10]
 [ 7 11]
 [ 7 12]
 [ 7 13]
 [ 8  0]
 [ 8  1]
 [ 8  2]
 [ 8  3]
 [ 8  4]
 [ 8  5]
 [ 8  6]
 [ 8  7]
 [ 8  8]
 [ 8  9]
 [ 8 10]
 [ 8 11]
 [ 8 12]
 [ 8 13]
 [ 8 14]
 [ 8 15]
 [ 8 16]
 [ 8 17]
 [ 8 18]
 [ 8 19]
 [ 8 20]
 [ 8 21]
 [ 8 

In [9]:
features

{'document_tokens': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x13623b8d0>,
 'query_tokens': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1364a0048>}